In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Sample_Data_Engineering_Assignment") \
    .getOrCreate()

spark


In [2]:
import os
os.getcwd()


'C:\\Users\\Lenovo'

In [3]:
import os

os.chdir(r"C:\Users\Lenovo\OneDrive\Documents\assigment\sample_dataset_Data_Engineering_Assignement")
os.getcwd()


'C:\\Users\\Lenovo\\OneDrive\\Documents\\assigment\\sample_dataset_Data_Engineering_Assignement'

In [4]:
os.listdir()


['.ipynb_checkpoints',
 'lengthbased_table.csv',
 'master_udm.csv',
 'Untitled.ipynb']

In [5]:
df_master = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("master_udm.csv")

df_length = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("lengthbased_table.csv")


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/C:/Users/Lenovo/master_udm.csv. SQLSTATE: 42K03

In [6]:
df_master = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(r"C:\Users\Lenovo\OneDrive\Documents\assigment\sample_dataset_Data_Engineering_Assignement\master_udm.csv")


In [7]:
df_length = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(r"C:\Users\Lenovo\OneDrive\Documents\assigment\sample_dataset_Data_Engineering_Assignement\lengthbased_table.csv")


In [8]:
df_master.count()
df_length.count()


2622798

In [9]:
df_master.printSchema()
df_length.printSchema()


root
 |-- Identifier: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Description_EN: string (nullable = true)
 |-- Description_Local: string (nullable = true)
 |-- Domain: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Type_Parquet: string (nullable = true)
 |-- Unit: string (nullable = true)
 |-- Range_Lower_Limit: string (nullable = true)
 |-- Range_Upper_Limit: string (nullable = true)
 |-- Sampling_Frequency: double (nullable = true)
 |-- Application: string (nullable = true)
 |-- Technology: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Asset_Tree: string (nullable = true)
 |-- Sensor_Position_Relative: string (nullable = true)
 |-- Sensor_Position_Absolute: string (nullable = true)
 |-- Asset_Position_Relative: string (nullable = true)
 |-- Sensor_Position_Top_Bottom: string (nullable = true)
 |-- Sensor_Position_Width: string (nullable = true)
 |-- Mqtt_Id: string (nullable = true)

root
 |-- TagId: string (nullable = 

In [10]:
df_master.show(5, truncate=False)
df_length.show(5, truncate=False)


+-------------------------------------------------------------------+--------------------------------------------------------+----------------------------------------------------------+----------------------------------------------------------+----------+----+------------+-----+-----------------+-----------------+------------------+-------------------------+----------+-------------+----------+------------------------+------------------------+-----------------------+--------------------------+---------------------+-------+
|Identifier                                                         |Name                                                    |Description_EN                                            |Description_Local                                         |Domain    |Type|Type_Parquet|Unit |Range_Lower_Limit|Range_Upper_Limit|Sampling_Frequency|Application              |Technology|Address      |Asset_Tree|Sensor_Position_Relative|Sensor_Position_Absolute|Asset_Position_Relative|Se

In [11]:
df_master.select("Identifier").distinct().count()


14180

In [12]:
df_length.select("TagId").distinct().count()


34

In [13]:
df_length.select("TagId").distinct() \
    .join(
        df_master.select("Identifier").distinct(),
        df_length.TagId == df_master.Identifier,
        "left_anti"
    ).show(truncate=False)


+-----+
|TagId|
+-----+
+-----+



In [14]:
df_length.filter("TagId IS NULL").count()


0

In [15]:
df_master.filter("Identifier IS NULL").count()


0

In [16]:
from pyspark.sql.functions import col

df_joined = (
    df_length.alias("l")
    .join(
        df_master.alias("m"),
        col("l.TagId") == col("m.Identifier"),
        how="left"
    )
)


In [17]:
df_length.count(), df_joined.count()


(2622798, 2622798)

In [18]:
df_joined.select(
    "TagId",
    "Name",
    "Unit",
    "Technology",
    "Application"
).show(10, truncate=False)


+----------------------------------+-----------------------+-----+----------+------------------+
|TagId                             |Name                   |Unit |Technology|Application       |
+----------------------------------+-----------------------+-----+----------+------------------+
|GN_CR_APL3_Bandsnelheid           |Bandsnelheid           |m/s  |PLC       |GN-CR-APL3_Energie|
|GN_CR_APL3_Oven_Debiet_gas_Z04_ist|Oven_Debiet_gas_Z04_ist|Nm3/h|PLC       |GN-CR-APL3_Energie|
|GN_CR_APL3_Oven_Debiet_gas_Z05_ist|Oven_Debiet_gas_Z05_ist|Nm3/h|PLC       |GN-CR-APL3_Energie|
|GN_CR_APL3_Oven_Debiet_gas_Z01_ist|Oven_Debiet_gas_Z01_ist|Nm3/h|PLC       |GN-CR-APL3_Energie|
|GN_CR_APL3_Oven_Debiet_gas_Z06_ist|Oven_Debiet_gas_Z06_ist|Nm3/h|PLC       |GN-CR-APL3_Energie|
|GN_CR_APL3_Oven_Debiet_gas_Z07_ist|Oven_Debiet_gas_Z07_ist|Nm3/h|PLC       |GN-CR-APL3_Energie|
|GN_CR_APL3_Oven_Debiet_gas_Z08_ist|Oven_Debiet_gas_Z08_ist|Nm3/h|PLC       |GN-CR-APL3_Energie|
|GN_CR_APL3_Oven_Debiet_gas_Z0